## Import and Plot the Network

In [1]:
import spatial_agg as sa
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
lines_df, nodes_df, wind_df, solar_df = sa.import_data()
sa.plot_network(lines_df, nodes_df);

## Test of Spatial Agg functions

In [11]:
TimeSeriesList = [solar_df, wind_df]
TimeHorizon = 24
CF = sa.create_capacity_factor_array(nodes_df, TimeSeriesList, TimeHorizon)
N_prime = 10
lambdas = [1,1,1]
N, G, T = CF.shape;


Number of nodes: 17
Number of time series features per node: 2
Time horizon: 24


In [ ]:
def correlation_features(CF):
    N, G, T = CF.shape
    TS_corr = np.zeros((N, N, G, G, T))

    for g in range(G):
        for h in range(G):
            if h != g:
                for n in range(N):
                    for m in range(N):
                        if m != n:
                            CF_n_g = CF[n, g, :]
                            CF_m_h = CF[m, h, :]
                            mean_n_g = np.mean(CF_n_g)
                            mean_m_h = np.mean(CF_m_h)
                            TS_corr[n, m, g, h, :] = (CF_n_g - mean_n_g) * (CF_m_h - mean_m_h)
    return TS_corr
    


In [ ]:
import numpy as np
import pandas as pd

def create_capacity_factor_array(nodes_df, TimeSeriesList, TimeHorizon):
    N = nodes_df.shape[0]
    T = TimeHorizon
    G = len(TimeSeriesList)
    CF = np.zeros((N, G, T))

    for g in range(G):
        for n in range(N):
            for t in range(T):
                CF[n, g, t] = TimeSeriesList[g].iloc[t, n]

    print(f"Number of nodes: {N}")
    print(f"Number of time series features per node: {G}")
    print(f"Time horizon: {T}")

    return CF


def correlation_features(CF):
    N, G, T = CF.shape
    TS_corr = np.zeros((N, N, G, G, T))

    for g in range(G):
        for h in range(G):
            if h != g:
                for n in range(N):
                    for m in range(N):
                        if m != n:
                            CF_n_g = CF[n, g, :]
                            CF_m_h = CF[m, h, :]
                            mean_n_g = np.mean(CF_n_g)
                            mean_m_h = np.mean(CF_m_h)
                            TS_corr[n, m, g, h, :] = (CF_n_g - mean_n_g) * (CF_m_h - mean_m_h)
    return TS_corr
    


# Assuming the functions correlation_features and create_capacity_factor_array are defined as provided

# Example usage:

# Step 1: Create the Capacity Factor Array
nodes_df = pd.DataFrame({
    'node_num': [0, 1, 2],
    'State': ['MA', 'MA', 'MA'],
    'Lat': [42.642711, 42.479477, 42.331960],
    'Lon': [-70.865107, -71.396507, -71.020173],
    'Offshore_wind_allowed': [1, 1, 1],
    'ISONE_zone': ['MA', 'MA', 'MA']
})

# Example TimeSeriesList with dummy data
TimeSeriesList = [
    pd.DataFrame(np.random.rand(24, 3)),  # Generator 1 time series
    pd.DataFrame(np.random.rand(24, 3))   # Generator 2 time series
]

TimeHorizon = 24

CF = create_capacity_factor_array(nodes_df, TimeSeriesList, TimeHorizon)

# Step 2: Calculate Correlation Features
TS_corr = correlation_features(CF)

# Step 3: Access Node Features
def get_node_features(node_index, CF, TS_corr, nodes_df):
    node_features = {}
    
    # Time series per generator at that node
    node_features['time_series'] = CF[node_index, :, :]
    
    # Position of the node
    node_features['position'] = (nodes_df.loc[node_index, 'Lat'], nodes_df.loc[node_index, 'Lon'])
    
    # Correlation time series between the generator at that node and any other generator at any other node
    N, G, T = CF.shape
    correlation_time_series = {}
    for g in range(G):
        for h in range(G):
            if h != g:
                for m in range(N):
                    if m != node_index:
                        correlation_time_series[(g, h, m)] = TS_corr[node_index, m, g, h, :]
    
    node_features['correlation_time_series'] = correlation_time_series
    
    return node_features

# Example: Get features for node 0
node_index = 0
node_features = get_node_features(node_index, CF, TS_corr, nodes_df)

print("Node Features:")
print("Time Series per Generator at Node:", node_features['time_series'])
print("Position of Node:", node_features['position'])
print("Correlation Time Series:", node_features['correlation_time_series'])

In [ ]:
def corr_coeff(CF, N, G, T):
    corr_coeff = np.zeros((N, N, G, G))
    for g in range(G):
        for h in range(G):
            if h != g:
                for n in range(N):
                    for m in range(N):
                        if m != n:
                            CF_n_g = CF[n, g, :]
                            CF_m_h = CF[m, h, :]
                            mean_n_g = np.mean(CF_n_g)
                            mean_m_h = np.mean(CF_m_h)
                            numerator = np.sum((CF_n_g - mean_n_g) * (CF_m_h - mean_m_h))
                            denominator = np.sqrt(np.sum((CF_n_g - mean_n_g)**2) * np.sum((CF_m_h - mean_m_h)**2))
                            corr_coeff[n, m, g, h] = numerator / denominator
    return corr_coeff

In [ ]:
# Initialize the model
model = gp.Model("node_aggregation")
model.Params.NonConvex = 2

# Define variables
alpha = model.addVars(N, N_prime, vtype=GRB.BINARY, name="alpha")
C_agg = model.addVars(
    N_prime, G, T, vtype=GRB.CONTINUOUS, name="C_agg", lb=0)
coords_agg = model.addVars(
    N_prime, 2, vtype=GRB.CONTINUOUS, name="coords_agg", lb=0)

# Objective components

### NRMSE
# Variable and Constraint to handle the square root in NRMSE_av
sqrt_NRMSE_terms = model.addVars(N, G, vtype=GRB.CONTINUOUS, name="sqrt_NRMSE_terms")
for n in range(N):
    for g in range(G):
        for n_prime in range(N_prime):
            squared_error = (1 / T) * sum((CF[n, g, t] - C_agg[n_prime, g, t])**2 for t in range(T))
            model.addConstr(sqrt_NRMSE_terms[n, g] * sqrt_NRMSE_terms[n, g] >= squared_error)
            
# Now define NRMSE_av using the auxiliary variable sqrt_NRMSE_terms
NRMSE_av = sum(sum(alpha[n, n_prime] * (1 / G) * sqrt_NRMSE_terms[n, g] / (CF[n, g, :].max() - CF[n, g, :].min())
                for g in range(G)) for n_prime in range(N_prime) for n in range(N))

### Spatial distance error
# Define auxiliary variables for spatial distance
dist = model.addVars(N, N_prime, vtype=GRB.CONTINUOUS, name="dist")

# Constraints to approximate the Euclidean distance
for n in range(N):
    for n_prime in range(N_prime):
        # Calculate the squared Euclidean distance between points
        squared_distance = (nodes_df.iloc[n]['Lat'] - coords_agg[n_prime, 0])**2 + \
                           (nodes_df.iloc[n]['Lon'] - coords_agg[n_prime, 1])**2

        # Constrain dist[n, n_prime] to be at least the square root of the squared distance
        model.addConstr(dist[n, n_prime] * dist[n, n_prime] >= squared_distance)

# Define the spatial distance error objective term using the auxiliary variable
spatial_distance_error = sum(sum(alpha[n, n_prime] * dist[n, n_prime] for n_prime in range(N_prime)) for n in range(N))

# Set objective
model.setObjective(lambdas[0] * NRMSE_av + lambdas[1] * spatial_distance_error, GRB.MINIMIZE)

# Add constraints
model.addConstrs((gp.quicksum(alpha[n, n_prime] for n_prime in range(
    N_prime)) == 1 for n in range(N)), "assignment")

# Optimize the model
model.optimize()

C_agg.x, alpha.x, coords_agg.x

Set parameter NonConvex to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 17 rows, 874 columns and 170 nonzeros
Model fingerprint: 0xfeb48fbb
Model has 510 quadratic objective terms
Model has 510 quadratic constraints
Variable types: 704 continuous, 170 integer (170 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [4e-02, 1e+00]
  QLMatrix range   [1e-05, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [6e-03, 7e+03]
Presolve time: 0.08s
Presolved: 1037 rows, 2404 columns, 2720 nonzeros
Presolved model has 1020 SOS constraint(s)
Presolved model has 510 quadratic constraint(s)
Variable types: 1724 continuous, 680 

In [28]:
# Initialize the model
model = gp.Model("node_aggregation")
model.Params.NonConvex = 2

# Define variables
alpha = model.addVars(N, N_prime, vtype=GRB.BINARY, name="alpha")
C_agg = model.addVars(
    N_prime, G, T, vtype=GRB.CONTINUOUS, name="C_agg", lb=0)
coords_agg = model.addVars(
    N_prime, 2, vtype=GRB.CONTINUOUS, name="coords_agg", lb=0)

# Objective components

### NRMSE
# Variable and Constraint to handle the square root in NRMSE_av
sqrt_NRMSE_terms = model.addVars(N, G, vtype=GRB.CONTINUOUS, name="sqrt_NRMSE_terms")
for n in range(N):
    for g in range(G):
        for n_prime in range(N_prime):
            squared_error = (1 / T) * sum((CF[n, g, t] - C_agg[n_prime, g, t])**2 for t in range(T))
            model.addConstr(sqrt_NRMSE_terms[n, g] * sqrt_NRMSE_terms[n, g] >= squared_error)
            
# Now define NRMSE_av using the auxiliary variable sqrt_NRMSE_terms
NRMSE_av = sum(sum(alpha[n, n_prime] * (1 / G) * sqrt_NRMSE_terms[n, g] / (CF[n, g, :].max() - CF[n, g, :].min())
                for g in range(G)) for n_prime in range(N_prime) for n in range(N))

### Correlation Error Term with Approximation
# Auxiliary variables for the correlation terms
numerator_terms = model.addVars(N, N, G, G, vtype=GRB.CONTINUOUS, name="numerator_terms")
denominator_terms = model.addVars(N, N, G, G, vtype=GRB.CONTINUOUS, name="denominator_terms")
approx_sqrt_denom = model.addVars(N, N, G, G, vtype=GRB.CONTINUOUS, name="approx_sqrt_denom")
abs_diff_corr = model.addVars(N, N, G, G, vtype=GRB.CONTINUOUS, name="abs_diff_corr")

for n in range(N):
    for m in range(N):
        if m != n:
            for g in range(G):
                for h in range(G):
                    if h != g:
                        # Mean values precomputed for correlation calculation
                        CF_n_g = CF[n, g, :]
                        CF_m_h = CF[m, h, :]
                        mean_n_g = np.mean(CF_n_g)
                        mean_m_h = np.mean(CF_m_h)

                        # Define numerator for correlation coefficient
                        numerator_expr = sum((CF_n_g[t] - mean_n_g) * (CF_m_h[t] - mean_m_h) for t in range(T))
                        model.addConstr(numerator_terms[n, m, g, h] == numerator_expr)

                        # Define denominator for correlation coefficient using auxiliary variables
                        sum_sq_n = sum((CF_n_g[t] - mean_n_g) ** 2 for t in range(T))
                        sum_sq_m = sum((CF_m_h[t] - mean_m_h) ** 2 for t in range(T))

                        # Ensure denominator_terms approximates sum of squares
                        model.addConstr(denominator_terms[n, m, g, h] == sum_sq_n * sum_sq_m)
                        model.addConstr(approx_sqrt_denom[n, m, g, h] * approx_sqrt_denom[n, m, g, h] >= denominator_terms[n, m, g, h])

                        # Absolute difference in correlation coefficients
                        corr_CF = numerator_terms[n, m, g, h] / (approx_sqrt_denom[n, m, g, h] + 1e-5)  # Small constant to avoid div by zero
                        corr_C_agg = sum((C_agg[n_prime, g, t] - mean_n_g) * (C_agg[m_prime, h, t] - mean_m_h) for t in range(T)) / (approx_sqrt_denom[n, m, g, h] + 1e-5)
                        model.addConstr(abs_diff_corr[n, m, g, h] >= corr_CF - corr_C_agg)
                        model.addConstr(abs_diff_corr[n, m, g, h] >= corr_C_agg - corr_CF)

corr_error = sum(sum(sum(sum(alpha[n, n_prime] * alpha[m, m_prime] / (G*(G-1)) * abs_diff_corr[n, m, g, h]
                            for h in range(G) if h != g) for g in range(G)) for m_prime in range(N_prime)) for n_prime in range(N_prime))



# NRMSE_av = sum(sum(alpha[n, n_prime] * (1 / G) * sum((1/T * sum((CF[n, g, t] - C_agg[n_prime, g, t])**2 for t in range(
#     T)))**0.5 / (CF[n, g, :].max() - CF[n, g, :].min()) for g in range(G)) for n_prime in range(N_prime)) for n in range(N))

# Set objective
model.setObjective(lambdas[0] * NRMSE_av + lambdas[1] * corr_error, GRB.MINIMIZE)

# Add constraints
model.addConstrs((gp.quicksum(alpha[n, n_prime] for n_prime in range(
    N_prime)) == 1 for n in range(N)), "assignment")
# model.addConstrs((C_agg[n_prime, :, :] * gp.quicksum(alpha[n, n_prime] for n in range(N)) == gp.quicksum(alpha[n, n_prime] * CF[n, :, :] for n in range(
#     N)) for n_prime in range(N_prime)), "capacity_factor")

# Optimize the model
model.optimize()

C_agg.x, alpha.x, coords_agg.x

Set parameter NonConvex to value 2


GurobiError: Divisor must be a constant

In [ ]:
# Initialize the model
model = gp.Model("node_aggregation")
model.Params.NonConvex = 2

# Define variables
alpha = model.addVars(N, N_prime, vtype=GRB.BINARY, name="alpha")
C_agg = model.addVars(
    N_prime, G, T, vtype=GRB.CONTINUOUS, name="C_agg", lb=0)
coords_agg = model.addVars(
    N_prime, 2, vtype=GRB.CONTINUOUS, name="coords_agg", lb=0)

# Objective components
# Additional step: Define auxiliary variables for square root terms
sqrt_NRMSE_terms = model.addVars(N, G, vtype=GRB.CONTINUOUS, name="sqrt_NRMSE_terms")

# Constraints to handle the square root in NRMSE_av
for n in range(N):
    for g in range(G):
        for n_prime in range(N_prime):
            # Define the squared error term
            squared_error = (1 / T) * sum((CF[n, g, t] - C_agg[n_prime, g, t])**2 for t in range(T))
            
            # Constrain the auxiliary variable to represent the square root of squared_error
            model.addConstr(sqrt_NRMSE_terms[n, g] * sqrt_NRMSE_terms[n, g] >= squared_error)
            
# Now define NRMSE_av using the auxiliary variable sqrt_NRMSE_terms
NRMSE_av = sum(sum(alpha[n, n_prime] * (1 / G) * sqrt_NRMSE_terms[n, g] / (CF[n, g, :].max() - CF[n, g, :].min())
                for g in range(G)) for n_prime in range(N_prime) for n in range(N))


NRMSE_av = sum(sum(alpha[n, n_prime] * (1 / G) * sum((1/T * sum((CF[n, g, t] - C_agg[n_prime, g, t])**2 for t in range(
    T)))**0.5 / (CF[n, g, :].max() - CF[n, g, :].min()) for g in range(G)) for n_prime in range(N_prime)) for n in range(N))

corr_error = sum(sum(sum(sum(alpha[n, n_prime] * alpha[m, m_prime] / (G*(G-1)) * sum(sum(gp.abs_(corr_coeff(CF)[n, m, g, h] - corr_coeff(C_agg)[n_prime, m_prime, g, h])
                    for h in range(G) if h != g) for g in range(G)) for m_prime in range(N_prime)) for n_prime in range(N_prime)) for m in range(N) if m != n) for n in range(N))

spatial_distance_error = sum(sum(alpha[n, n_prime] * ((nodes_df.iloc[n]['Lat'] - coords_agg[n_prime, 0])**2 + (
    nodes_df.iloc[n]['Lon'] - coords_agg[n_prime, 1])**2)**0.5 for n_prime in range(N_prime)) for n in range(N))

# Set objective
model.setObjective(lambdas[0] * NRMSE_av + lambdas[3] * corr_error +
                    lambdas[2] * spatial_distance_error, GRB.MINIMIZE)

# Add constraints
model.addConstrs((gp.quicksum(alpha[n, n_prime] for n_prime in range(
    N_prime)) == 1 for n in range(N)), "assignment")
model.addConstrs((C_agg[n_prime, :, :] * gp.quicksum(alpha[n, n_prime] for n in range(N)) == gp.quicksum(alpha[n, n_prime] * CF[n, :, :] for n in range(
    N)) for n_prime in range(N_prime)), "capacity_factor")

# Optimize the model
model.optimize()

return CF.x, alpha.x, coords_agg.x

In [ ]:
N_prime = 10
lambdas = [1, 1, 1]
N = CF.shape[0]
G = CF.shape[1]
T = CF.shape[2]
alpha = np.random.rand(N, N_prime)
C_agg = np.random.rand(N_prime, G, T)

NRMSE_av = sum(sum(alpha[n, n_prime] * (1 / G) * sum(np.sqrt(1/T * sum((CF[n, g, t] - C_agg[n_prime, g, t])**2 for t in range(T))) / (CF[n, g, :].max() - CF[n, g, :].min()) for g in range(G)) for n_prime in range(N_prime)) for n in range(N))


In [ ]:
N_prime = 10
lambda1, lambda2, lambda3 = 1, 1, 1
CF_result, alpha_result, coords_result = sa.aggregate_nodes(CF, nodes_df, N_prime)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06


Set parameter NonConvex to value 2


TypeError: unsupported operand type(s) for ** or pow(): 'gurobipy.QuadExpr' and 'float'

In [ ]:
corr_coeff = np.zeros((N, N, G, G))
corr_coeff1 = np.zeros((N, N, G, G))
corr_coeff2 = np.zeros((N, N, G, G))

for g in range(G):
    for h in range(G):
        if h != g:
            CF_g = CF[:, g, :]
            CF_h = CF[:, h, :]
            corr_matrix = np.corrcoef(CF_g, CF_h, rowvar=True)
            corr_coeff2[:, :, g, h] = corr_matrix[:N, N:]

for g in range(G):
    for h in range(G):
        if h != g:
            for n in range(N):
                for m in range(N):
                    if m != n:
                        CF_n_g = CF[n, g, :]
                        CF_m_h = CF[m, h, :]
                        mean_n_g = np.mean(CF_n_g)
                        mean_m_h = np.mean(CF_m_h)
                        numerator = np.sum((CF_n_g - mean_n_g) * (CF_m_h - mean_m_h))
                        denominator = np.sqrt(np.sum((CF_n_g - mean_n_g)**2) * np.sum((CF_m_h - mean_m_h)**2))
                        corr_coeff[n, m, g, h] = numerator / denominator
                        corr_coeff1[n, m, g, h] = np.corrcoef(CF_n_g, CF_m_h)[0, 1]
                        if not np.isclose(corr_coeff[n, m, g, h], corr_coeff1[n, m, g, h], atol=1e-8):
                            print(f"Difference at g={g}, h={h}, n={n}, m={m}: corr_coeff={corr_coeff[n, m, g, h]}, corr_coeff1={corr_coeff1[n, m, g, h]}")
                        if not np.isclose(corr_coeff2[n, m, g, h], corr_coeff[n, m, g, h], atol=1e-15):
                            print(f"Difference at g={g}, h={h}, n={n}, m={m}: corr_coeff2={corr_coeff2[n, m, g, h]}, corr_coeff={corr_coeff[n, m, g, h]}")
        elif g == h:
            for n in range(N):
                for m in range(N):
                    if m != n:
                        print (f"n={n}, m={m}, g={g}, h={h}: corr_coeff={corr_coeff[n, m, g, h]}, corr_coeff1={corr_coeff1[n, m, g, h]}, corr_coeff2={corr_coeff2[n, m, g, h]}")
                        if not np.isclose(corr_coeff2[n, m, g, h], corr_coeff[n, m, g, h], atol=1e-15):
                            print(f"Difference at g={g}, h={h}, n={n}, m={m}: corr_coeff2={corr_coeff2[n, m, g, h]}, corr_coeff={corr_coeff[n, m, g, h]}")
                    else:
                        print (f"n={n}, m={m}, g={g}, h={h}: corr_coeff={corr_coeff[n, m, g, h]}, corr_coeff1={corr_coeff1[n, m, g, h]}, corr_coeff2={corr_coeff2[n, m, g, h]}")
                        if not np.isclose(corr_coeff2[n, m, g, h], corr_coeff[n, m, g, h], atol=1e-15):
                            print(f"Difference at g={g}, h={h}, n={n}, m={m}: corr_coeff2={corr_coeff2[n, m, g, h]}, corr_coeff={corr_coeff[n, m, g, h]}")
        


n=0, m=0, g=0, h=0: corr_coeff=0.0, corr_coeff1=0.0, corr_coeff2=1.0
Difference at g=0, h=0, n=0, m=0: corr_coeff2=1.0, corr_coeff=0.0
n=0, m=1, g=0, h=0: corr_coeff=0.0, corr_coeff1=0.0, corr_coeff2=0.9904004755094789
Difference at g=0, h=0, n=0, m=1: corr_coeff2=0.9904004755094789, corr_coeff=0.0
n=0, m=2, g=0, h=0: corr_coeff=0.0, corr_coeff1=0.0, corr_coeff2=0.9830625979292967
Difference at g=0, h=0, n=0, m=2: corr_coeff2=0.9830625979292967, corr_coeff=0.0
n=0, m=3, g=0, h=0: corr_coeff=0.0, corr_coeff1=0.0, corr_coeff2=0.9511467652280147
Difference at g=0, h=0, n=0, m=3: corr_coeff2=0.9511467652280147, corr_coeff=0.0
n=0, m=4, g=0, h=0: corr_coeff=0.0, corr_coeff1=0.0, corr_coeff2=0.9381865853547733
Difference at g=0, h=0, n=0, m=4: corr_coeff2=0.9381865853547733, corr_coeff=0.0
n=0, m=5, g=0, h=0: corr_coeff=0.0, corr_coeff1=0.0, corr_coeff2=0.8776534427980515
Difference at g=0, h=0, n=0, m=5: corr_coeff2=0.8776534427980515, corr_coeff=0.0
n=0, m=6, g=0, h=0: corr_coeff=0.0, corr

In [2]:
import spatial_agg_objoriented as saoo
import spatial_agg as sa

lines_df, nodes_df, wind_df, solar_df = sa.import_data()
time_series_dict = {
     'wind': wind_df,
     'solar': solar_df
 }
TimeHorizon = 24
CF = sa.create_capacity_factor_array(nodes_df, TimeSeriesList, TimeHorizon)
N_prime = 10
lambdas = [1,1,1]
N, G, T = CF.shape;


network = saoo.Network(nodes_df, time_series_dict, lines_df, TimeHorizon)
network.display_node_features(0)

# optimizer = AggregationOptimizer(network, target_node_count=2)
# optimizer.optimize_aggregation()
# optimizer.display_aggregated_network()

NameError: name 'TimeSeriesList' is not defined